In [1]:
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
X = np.array([[1, 2], [3, 4], [1, 2], [3, 4], [1, 2], [3, 4]])
y = np.array([1, 2, 3, 4, 5, 6])
tscv = TimeSeriesSplit()
print(tscv)
TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None)
for i, (train_index, test_index) in enumerate(tscv.split(X)):
    print(f"Fold {i}:")
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={test_index}")


TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None)
Fold 0:
  Train: index=[0]
  Test:  index=[1]
Fold 1:
  Train: index=[0 1]
  Test:  index=[2]
Fold 2:
  Train: index=[0 1 2]
  Test:  index=[3]
Fold 3:
  Train: index=[0 1 2 3]
  Test:  index=[4]
Fold 4:
  Train: index=[0 1 2 3 4]
  Test:  index=[5]


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import TensorBoard
import datetime
import os

# Налаштування даних
seq_length = 50
pred_length = 5
n_features = 12

# Імітація датасету
data_length = 5000  # Довжина датасету
data = np.random.rand(data_length, n_features)  # Випадкові дані для демонстрації

# Функція для створення послідовностей
def create_sequences(data, seq_length, pred_length):
    X, y = [], []
    for i in range(len(data) - seq_length - pred_length + 1):
        X.append(data[i:i+seq_length])  # Вхідна послідовність
        y.append(data[i+seq_length:i+seq_length+pred_length, 5])  # Прогнозування "oc_range" (індекс 5)
    return np.array(X), np.array(y)

# Масштабування даних
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Генеруємо послідовності
X, y = create_sequences(scaled_data, seq_length, pred_length)

# Налаштування TensorBoard
log_dir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Кастомна функція втрат
def custom_loss(y_true, y_pred):
    mse = tf.reduce_mean(tf.square(y_true - y_pred), axis=-1)
    return mse

# Побудова LSTM моделі без Attention, з візуалізацією сили фічей
def build_lstm_model(seq_length, n_features, pred_length, l2_value=0.0005, dropout_rate=0.2):
    input_seq = Input(shape=(seq_length, n_features))
    
    # Перший LSTM шар із Dropout
    x = LSTM(50, return_sequences=True, kernel_regularizer=l2(l2_value))(input_seq)
    x = Dropout(dropout_rate)(x)
    
    # Другий LSTM шар із Dropout
    x = LSTM(50, return_sequences=False, kernel_regularizer=l2(l2_value))(x)
    x = Dropout(dropout_rate)(x)
    
    # Вихідний шар
    output = Dense(pred_length)(x)
    
    # Створення та компіляція моделі
    model = Model(inputs=input_seq, outputs=output)
    model.compile(optimizer='adam', loss=custom_loss)
    return model

# Використання TimeSeriesSplit
n_splits = 5
tscv = TimeSeriesSplit(n_splits=n_splits)

for fold, (train_idx, test_idx) in enumerate(tscv.split(X)):
    print(f"Fold {fold + 1}/{n_splits}")
    
    # Розбиття на тренувальні та тестові набори
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    # Побудова моделі
    model = build_lstm_model(seq_length=seq_length, n_features=n_features, pred_length=pred_length)
    
    # Навчання моделі з TensorBoard
    model.fit(
        X_train, y_train, 
        epochs=10, 
        batch_size=32, 
        validation_data=(X_test, y_test),
        callbacks=[tensorboard_callback],
        verbose=1
    )
    
    # Оцінка моделі
    loss = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Loss for Fold {fold + 1}: {loss}\n")

# Візуалізація сили фічей у TensorBoard
@tf.function
def log_feature_importance(model, X_sample):
    """Обчислення ваг фічей і запис у TensorBoard."""
    with tf.GradientTape() as tape:
        tape.watch(X_sample)
        preds = model(X_sample)
    
    gradients = tape.gradient(preds, X_sample)
    feature_importance = tf.reduce_mean(tf.abs(gradients), axis=(0, 1))
    
    # Логування ваг фічей
    tf.summary.scalar("Feature Importance", data=tf.reduce_sum(feature_importance), step=1)
    for i, importance in enumerate(feature_importance):
        tf.summary.scalar(f"Feature_{i}_Importance", data=importance, step=1)

# Зразок даних для візуалізації ваг
sample_data = tf.convert_to_tensor(X[:1], dtype=tf.float32)  # Візьмемо першу послідовність
log_dir_features = os.path.join("logs", "features")
feature_writer = tf.summary.create_file_writer(log_dir_features)

# Логування ваг
with feature_writer.as_default():
    log_feature_importance(model, sample_data)

# Запуск TensorBoard:
print(f"To view TensorBoard logs, run:\n\ntensorboard --logdir={log_dir}")
print(f"To view feature importance logs, run:\n\ntensorboard --logdir={log_dir_features}")


Fold 1/5
Epoch 1/10
26/26 [==============================] - 7s 63ms/step - loss: 0.1802 - val_loss: 0.1269
Epoch 2/10
26/26 [==============================] - 1s 23ms/step - loss: 0.1367 - val_loss: 0.1207
Epoch 3/10
26/26 [==============================] - 1s 25ms/step - loss: 0.1288 - val_loss: 0.1145
Epoch 4/10
26/26 [==============================] - 1s 26ms/step - loss: 0.1246 - val_loss: 0.1145
Epoch 5/10
26/26 [==============================] - 1s 23ms/step - loss: 0.1180 - val_loss: 0.1071
Epoch 6/10
26/26 [==============================] - 1s 23ms/step - loss: 0.1137 - val_loss: 0.1034
Epoch 7/10
26/26 [==============================] - 1s 24ms/step - loss: 0.1109 - val_loss: 0.1031
Epoch 8/10
26/26 [==============================] - 1s 23ms/step - loss: 0.1072 - val_loss: 0.0992
Epoch 9/10
26/26 [==============================] - 1s 23ms/step - loss: 0.1067 - val_loss: 0.0983
Epoch 10/10
26/26 [==============================] - 1s 25ms/step - loss: 0.1036 - val_loss: 0.0966


OperatorNotAllowedInGraphError: in user code:

    File "C:\Users\Pc\AppData\Local\Temp\ipykernel_15876\85512661.py", line 106, in log_feature_importance  *
        for i, importance in enumerate(feature_importance):

    OperatorNotAllowedInGraphError: Iterating over a symbolic `tf.Tensor` is not allowed: AutoGraph did convert this function. This might indicate you are trying to use an unsupported feature.


In [3]:
@tf.function
def log_feature_importance(model, X_sample, writer):
    """Обчислення ваг фічей і запис у TensorBoard."""
    with tf.GradientTape() as tape:
        tape.watch(X_sample)
        preds = model(X_sample)

    gradients = tape.gradient(preds, X_sample)
    feature_importance = tf.reduce_mean(tf.abs(gradients), axis=(0, 1))

    # Запис ваг через TensorBoard
    with writer.as_default():
        tf.summary.scalar("Total Feature Importance", data=tf.reduce_sum(feature_importance), step=0)
        for i in tf.range(tf.shape(feature_importance)[0]):
            tf.summary.scalar(f"Feature_{i}_Importance", data=feature_importance[i], step=0)


In [4]:
# Логування ваг через TensorBoard
sample_data = tf.convert_to_tensor(X[:1], dtype=tf.float32)  # Зразок даних
log_dir_features = os.path.join("logs", "features")
feature_writer = tf.summary.create_file_writer(log_dir_features)

# Викликаємо функцію для логування
log_feature_importance(model, sample_data, feature_writer)

# Запуск TensorBoard
print(f"To view feature importance logs, run:\n\ntensorboard --logdir={log_dir_features}")


To view feature importance logs, run:

tensorboard --logdir=logs\features


In [6]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1
